In [ ]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "SECERET_KEY"

In [ ]:
!pip install -q langchain langchain_core langchain_huggingface requests

In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
import requests

# **Tools**

In [ ]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency : str, target_currency : str) -> float :
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """

  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()


@tool
def convert(base_currency_value : float, conversion_rate : Annotated[float, InjectedToolArg]) -> float :
  """
    This function take the base currency value with the conversion rate and returen the product of it
  """
  return (base_currency_value * conversion_rate)




## **Binding with LLM**

In [ ]:

hf_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

llm = HuggingFaceEndpoint (
    repo_id="Qwen/Qwen3-8B",
    task="text-generation",
    temperature=0.7,
    max_new_tokens=256,
    huggingfacehub_api_token=hf_token
)

model = ChatHuggingFace(llm=llm)

llm_with_tools = model.bind_tools([convert, get_conversion_factor])

In [ ]:
query = HumanMessage("what is the conversion factor between USD and INR, and based on that can you convert 10 USD to INR")

messages = [query]

In [ ]:
first_ai_message = llm_with_tools.invoke(messages)
messages.append(first_ai_message)


Model Qwen/Qwen3-8B is in staging mode for provider fireworks-ai. Meant for test purposes only.


In [ ]:
first_ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_1CwfRUtlMHYqOKvk4a4t0pop',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': 'call_wmy0GaG5ym8yLOk62juvFTSn',
  'type': 'tool_call'}]

In [ ]:
import json

for tool_call in first_ai_message.tool_calls :
  # tool 1
  if tool_call['name'] == 'get_conversion_factor' :
    tool_message1 = get_conversion_factor.invoke(tool_call)

    #fetch the conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    #append this tool message to message list
    messages.append(tool_message1)

  #tool 2
  if tool_call['name'] == 'convert' :
    #fetch the current arguments
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)

In [ ]:
result = llm_with_tools.invoke(messages)


Model Qwen/Qwen3-8B is in staging mode for provider fireworks-ai. Meant for test purposes only.


In [ ]:
result.content

"<think>\nOkay, let's break this down. The user asked for two things: the conversion factor between USD and INR, and converting 10 USD to INR.\n\nFirst, I called the get_conversion_factor function with USD and INR. The response gave a conversion rate of 88.7526. That's the factor needed for conversion.\n\nNext, to convert 10 USD to INR, I used the convert function. The user's initial tool definition for the convert function only had base_currency_value as a parameter. But in the second tool call, I added a conversion_rate parameter, which wasn't in the original tool definition. Wait, that might be an error. The original convert function didn't have conversion_rate as a parameter. But the user's second tool call included it. Maybe that's a mistake. However, the response from the tool gave 887.526, which is 10 multiplied by 88.7526. So even though the tool's parameters didn't include conversion_rate, the function might have used it internally. \n\nBut according to the original tool defin